In [ ]:
""" This module prepares midi file data and feeds it to the neural
network for training """

#import os
#os.environ['CUDA_VISIBLE_DEVICES']='1' 





import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#from keras.models import Sequential
#from keras.layers import Dense


from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm


#from keras.layers import Dropout
#from keras.layers import LSTM
#from keras.layers import Activation
#from keras.layers import BatchNormalization as BatchNorm



from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

#여기부터
#from keras.models import Sequential
#from keras.layers import Dense
#model = Sequential()
#model.add(Dense(4, input_dim=4, activation='relu'))
#model.add(Dense(3, activation='softmax'))



#from keras.utils.training_utils import multi_gpu_model
#model = multi_gpu_model(model, gpus=4)


#multi

# from __future__ import print_function

# import keras

# from keras.datasets import cifar10

# from keras.preprocessing.image import ImageDataGenerator

# from keras.models import Sequential

# from keras.layers import Dense, Dropout, Activation, Flatten

# from keras.layers import Conv2D, MaxPooling2D

# from tensorflow.keras.utils import multi_gpu_model

# import os


# from keras.utils.training_utils import multi_gpu_model
# #from keras.utils.training_utils import multi_gpu_model
# model = multi_gpu_model(model, gpus=4)

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


#model = keras.utils.multi_gpu_model(model, gpus=4)
#model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(lr=1e-4), metrics=['acc'])
# 여기까지


#tf.compat.v1.disable_eager_execution()

'''
tf.debugging.set_log_device_placement(True)


physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:

    tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.set_visible_devices(physical_devices[0:1], 'GPU')
'''




#gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_memory_growth(gpus[0], True)
# except RuntimeError as e:
#     # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
#      print(e)

    
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.python.keras import backend as K

# #from keras.backend import tensorflow_backend as K
# #config = tf.ConfigProto()
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# #K.set_session(tf.Session(config=config))
# K.set_session(tf.compat.v1.Session(config=config))

# import tensorflow as tf

# #config = tf.ConfigProto()
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4

# #session = tf.Session(config=config)
# session = tf.compat.v1.Session(config=config)



#session.close()

    
    



def train_network():
    #tf.debugging.set_log_device_placement(True)
   # with tf.device('/device:GPU:2'):

    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    #tf.debugging.set_log_device_placement(True)
  #  with tf.device('/device:GPU:2'):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,  # recurrent_dropout=0.3,
        return_sequences=True
    ))

    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))  #recurrent_dropout=0.3
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))   # model.add(Dropout(0.3))  
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))    # model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()


Parsing midi_songs\0fithos.mid
Parsing midi_songs\8.mid
Parsing midi_songs\ahead_on_our_way_piano.mid
Parsing midi_songs\AT.mid
Parsing midi_songs\balamb.mid
Parsing midi_songs\bcm.mid
Parsing midi_songs\BlueStone_LastDungeon.mid
Parsing midi_songs\braska.mid
Parsing midi_songs\caitsith.mid
Parsing midi_songs\Cids.mid
Parsing midi_songs\cosmo.mid
Parsing midi_songs\costadsol.mid
Parsing midi_songs\dayafter.mid
Parsing midi_songs\decisive.mid
Parsing midi_songs\dontbeafraid.mid
Parsing midi_songs\DOS.mid
Parsing midi_songs\electric_de_chocobo.mid
Parsing midi_songs\Eternal_Harvest.mid
Parsing midi_songs\EyesOnMePiano.mid
Parsing midi_songs\ff11_awakening_piano.mid
Parsing midi_songs\ff1battp.mid
Parsing midi_songs\FF3_Battle_(Piano).mid
Parsing midi_songs\FF3_Third_Phase_Final_(Piano).mid
Parsing midi_songs\ff4-airship.mid
Parsing midi_songs\Ff4-BattleLust.mid
Parsing midi_songs\ff4-fight1.mid
Parsing midi_songs\ff4-town.mid
Parsing midi_songs\FF4.mid
Parsing midi_songs\ff4pclov.mid
Par

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# allow_growth

# 해당 옵션을 True로 설정할 경우, 아래와 같이 GPU의 메모리가 전부 할당되지 않고, 아주 적은 비율만 할당되어 시작해서, 프로세스의 메모리 수요에 따라 증가하게 된다. 


import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.python.keras import backend as K

#from keras.backend import tensorflow_backend as K
#config = tf.ConfigProto()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
#K.set_session(tf.Session(config=config))
K.set_session(tf.compat.v1.Session(config=config))


# per_process_gpu_memory_fraction
# 해당 옵션은 원하는 gpu에 어느 비율의 메모리를 할당할지 먼저 결정하는 것으로, 프로세스가 어느정도의 메모리를 사용할지 알고 있다면, 작업 전에 미리 지정하여 메모리를 아낄 수 있다.


import tensorflow as tf

#config = tf.ConfigProto()
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4

#session = tf.Session(config=config)
session = tf.compat.v1.Session(config=config)
session.close()


In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18145463359412617160,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9873752679
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14683852620828401655
 physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())